In [3]:
import os

In [4]:
%pwd

'c:\\Users\\knadim\\Desktop\\MLProjects\\Kidney-Disease-Classification\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\knadim\\Desktop\\MLProjects\\Kidney-Disease-Classification'

In [29]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub


In [ ]:
#### Find where is the cacert.pem file to add the SSL certifcate of dagshub
#### Then we need to follow the steps to add the certifcate
import certifi
print(certifi.where())


c:\Users\knadim\.conda\envs\kidney\Lib\site-packages\certifi\cacert.pem


In [ ]:
#### Validate that the certifcate works 

import requests

# Test HTTPS connection to Dagshub
response = requests.get("https://dagshub.com")
print(response.status_code)

In [30]:
class Evaluation:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.mlflow_uri = "https://dagshub.com/karim-nadim/Kidney-Disease-Classification.mlflow"
        
        create_directories([self.config.artifacts_root])

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.params.IMAGE_SIZE[:-1],
            batch_size=self.params.BATCH_SIZE,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=Path(os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")),
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.training.trained_model_path)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        dagshub.init(repo_owner='karim-nadim', repo_name='Kidney-Disease-Classification', mlflow=True)

        with mlflow.start_run():
            mlflow.log_params(self.params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [31]:
try:
    evaluation = Evaluation()
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-09-17 14:21:40,264: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-17 14:21:40,264: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-17 14:21:40,280: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 5s 540ms/step - loss: 0.0766 - accuracy: 0.9568
[2025-09-17 14:21:45,916: INFO: common: json file saved at: scores.json]
[2025-09-17 14:21:46,162: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/karim-nadim/Kidney-Disease-Classification "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "karim-nadim/Kidney-Disease-Classification"

[2025-09-17 14:21:46,168: INFO: helpers: Initialized MLflow to track repo "karim-nadim/Kidney-Disease-Classification"]


Repository karim-nadim/Kidney-Disease-Classification initialized!

[2025-09-17 14:21:46,171: INFO: helpers: Repository karim-nadim/Kidney-Disease-Classification initialized!]


2025/09/17 14:21:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-09-17 14:21:48,014: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\knadim\AppData\Local\Temp\tmpienrtvjj\model\data\model\assets
[2025-09-17 14:21:48,624: INFO: builder_impl: Assets written to: C:\Users\knadim\AppData\Local\Temp\tmpienrtvjj\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/09/17 14:22:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
